# 读入keras五个模型的特征参数文件

In [1]:
root_src = 'D:\Dogs vs. Cats'
import os
import shutil
import time
os.chdir(root_src)
import numpy as np
import h5py
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5","gap_VGG16.h5","gap_VGG19.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])



# 打印特征参数的大小

In [2]:
for i in range(5):
    print(X_train[i].shape)

(25000, 2048)
(25000, 2048)
(25000, 2048)
(25000, 512)
(25000, 512)


In [3]:
from keras.models import *
from keras.layers import *
from keras.callbacks import EarlyStopping
import pandas as pd
from keras.preprocessing.image import *

Using TensorFlow backend.


# 列出五个模型组合的可能结果（31种）

In [4]:

               #only one model
merge_sequece = [[1,0,0,0,0],
                 [0,1,0,0,0],
                 [0,0,1,0,0],   
                 [0,0,0,1,0],
                 [0,0,0,0,1],
                 #two model
                 [1,1,0,0,0],
                 [1,0,1,0,0],  
                 [1,0,0,1,0],
                 [1,0,0,0,1],
                 [0,1,1,0,0],
                 [0,1,0,1,0],
                 [0,1,0,0,1],
                 [0,0,1,1,0],
                 [0,0,1,0,1],
                 [0,0,0,1,1],
                 #three model
                 [1,1,1,0,0],
                 [1,1,0,1,0],
                 [1,1,0,0,1],
                 [1,0,1,1,0],
                 [1,0,1,0,1],
                 [1,0,0,1,1],
                 [0,1,1,0,1],
                 [0,1,1,1,0],
                 [0,1,0,1,1],
                 [0,0,1,1,1],
                 #four model
                 [1,1,1,1,0],
                 [1,1,1,0,1],
                 [1,1,0,1,1],
                 [1,0,1,1,1],
                 [0,1,1,1,1],
                 #one model
                 [1,1,1,1,1]]

# 开始训练，测试，并保存结果，便于上传至Kaggle

In [5]:
import time
start_time = time.time()
for i,data_idx in enumerate(merge_sequece):
    print('test model merge:%d'%i)
    #find merge model        
    merge_idx = [k for k, x in enumerate(data_idx) if x == 1]
    print(merge_idx) 
    X_train_merge = [ ]
    X_test_merge  = [ ]
    y_train_merge = y_train
    for idx in merge_idx:
            #print(idx)
            X_train_merge.append( X_train[idx])
            X_test_merge.append(X_test[idx])
    
    X_train_merge = np.concatenate(X_train_merge, axis=1)
    X_test_merge = np.concatenate(X_test_merge, axis=1)

    X_train_merge, y_train_merge = shuffle(X_train_merge, y_train_merge)
    
    print('X_train_merge.shape' , '    ', 'X_test_merge.shape','    ','y_train_merge.shape')
    print(X_train_merge.shape,'           ', X_test_merge.shape, '          ', y_train_merge.shape)

    input_tensor = Input(X_train_merge.shape[1:])
    # 可能会有部分特征一致或者类似，所以dropout一些特征，另一方面也可以防止过拟合
    x = Dropout(0.25)(input_tensor)
    x = Dense(1, activation='sigmoid')(x)#直接设置全连接
    model = Model(input_tensor, x)

    model.compile(optimizer='adadelta',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=1, verbose=0, mode='auto')
    model.fit(X_train_merge, y_train_merge, batch_size=128, epochs=10, validation_split=0.2,callbacks=[earlystopping])
    
    y_pred = model.predict(X_test_merge, verbose=1)
    y_pred = y_pred.clip(min=0.005, max=0.995)

    df = pd.read_csv("sample_submission.csv")

    gen = ImageDataGenerator()
    test_generator = gen.flow_from_directory("test", (224, 224), shuffle=False, 
                                             batch_size=16, class_mode=None)
    # 获取文件的序号并减1，然后设置到新预测的csv文件的序号
    for m, fname in enumerate(test_generator.filenames):
        index = int((fname.split('\\')[1]).split('.')[0])
        df.set_value(index-1, 'label', y_pred[m])
    merge_file = 'merge_sample_submission'+str(i)+'.csv'
    print('create merge file',str(i),':',merge_file)
    df.to_csv(merge_file, index=None)
    
    #df.head(10)
end_time = time.time()
print('total cost time is %d s'%round(end_time- start_time))

test model merge:0
[0]
X_train_merge.shape      X_test_merge.shape      y_train_merge.shape
(25000, 2048)             (12500, 2048)            (25000,)
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 4s - loss: 0.2475 - acc: 0.9023 - val_loss: 0.1071 - val_acc: 0.9686
Epoch 2/10
20000/20000 [==============================] - 2s - loss: 0.1034 - acc: 0.9627 - val_loss: 0.0776 - val_acc: 0.9728
Epoch 3/10
20000/20000 [==============================] - 2s - loss: 0.0805 - acc: 0.9695 - val_loss: 0.0702 - val_acc: 0.9732
Epoch 4/10
20000/20000 [==============================] - 2s - loss: 0.0714 - acc: 0.9723 - val_loss: 0.0617 - val_acc: 0.9788
Epoch 5/10
20000/20000 [==============================] - 2s - loss: 0.0675 - acc: 0.9736 - val_loss: 0.0592 - val_acc: 0.9788
Epoch 6/10
20000/20000 [==============================] - 2s - loss: 0.0632 - acc: 0.9751 - val_loss: 0.0577 - val_acc: 0.9790
Epoch 7/10
20000/20000 [=============